In [9]:
!pip install python-binance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 55.3 MB/s eta 0:00:00


In [1]:
# Google Colab Test Notebook for AI Financial Forecasting System
# This notebook tests the core functionality without requiring all external APIs

# Cell 1: Install Dependencies
!pip install yfinance pandas numpy scikit-learn matplotlib seaborn plotly
!pip install prophet statsmodels tensorflow keras
!pip install textblob vaderSentiment
!pip install python-dotenv pyyaml
!pip install openai  # You'll need your OpenAI API key
print("✅ Dependencies installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.8 MB/s eta 0:00:00
✅ Dependencies installed!


In [18]:
# Cell 2: Set up environment variables using Google Colab Secrets
import os
from google.colab import userdata

# Access API keys from Colab Secrets
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY') # Get from https://platform.openai.com/api-keys
os.environ['FRED_API_KEY'] = userdata.get('FRED_API_KEY')      # Get from https://fred.stlouisfed.org/docs/api/api_key.html
os.environ['NEWS_API_KEY'] = userdata.get('NEWS_API_KEY')      # Get from https://newsapi.org/register

# Optional - Reddit API (for better sentiment analysis)
os.environ['REDDIT_CLIENT_ID'] = userdata.get('REDDIT_CLIENT_ID')
os.environ['REDDIT_CLIENT_SECRET'] = userdata.get('REDDIT_CLIENT_SECRET')
os.environ['REDDIT_USER_AGENT'] = userdata.get('REDDIT_USER_AGENT')
print("✅ Environment variables set from Colab Secrets!")

✅ Environment variables set from Colab Secrets!


In [19]:
# COMPLETE AI FINANCIAL FORECASTING SYSTEM - SIMPLIFIED VERSION
# This is a clean, working implementation that consolidates all your agents

import asyncio
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Any
import warnings
warnings.filterwarnings('ignore')

# Optional imports (install if needed)
try:
    from prophet import Prophet
    HAS_PROPHET = True
except ImportError:
    HAS_PROPHET = False
    print("⚠️ Prophet not available - using simple forecasting")

try:
    from openai import OpenAI
    HAS_OPENAI = True
except ImportError:
    HAS_OPENAI = False
    print("⚠️ OpenAI not available - using rule-based recommendations")

# =============================================================================
# DATA STRUCTURES
# =============================================================================

@dataclass
class MarketData:
    symbol: str
    current_price: float
    prices: pd.Series
    volume: pd.Series = field(default_factory=pd.Series)

    # Technical indicators
    rsi: float = 50.0
    trend: str = "neutral"
    return_1d: float = 0.0
    return_5d: float = 0.0
    return_20d: float = 0.0
    volatility_20d: float = 0.0
    support_level: float = 0.0
    resistance_level: float = 0.0

    last_updated: datetime = field(default_factory=datetime.now)

@dataclass
class ForecastData:
    arima_forecast: float = 0.0
    prophet_forecast: float = 0.0
    lstm_forecast: float = 0.0
    ensemble_forecast: float = 0.0
    forecast_confidence: float = 0.5
    prediction_interval: List[float] = field(default_factory=lambda: [0.0, 0.0])
    forecast_horizon_days: int = 5
    forecast_accuracy_score: float = 0.0
    last_updated: datetime = field(default_factory=datetime.now)

@dataclass
class MacroData:
    gdp_growth: float = 0.0
    inflation_rate: float = 0.0
    unemployment_rate: float = 0.0
    federal_funds_rate: float = 0.0
    vix: float = 0.0
    dollar_index: float = 0.0
    market_sentiment: str = "neutral"
    last_updated: datetime = field(default_factory=datetime.now)

@dataclass
class SentimentData:
    news_sentiment: float = 0.0
    social_media_sentiment: float = 0.0
    overall_sentiment: float = 0.0
    sentiment_trend: str = "neutral"
    confidence_score: float = 0.5
    key_topics: List[str] = field(default_factory=list)
    last_updated: datetime = field(default_factory=datetime.now)

@dataclass
class TradingRecommendation:
    action: str  # "BUY", "SELL", "HOLD"
    confidence: float
    position_size: float
    entry_price: float
    stop_loss: float
    take_profit: float
    risk_level: str
    reasoning: str
    time_horizon: str
    last_updated: datetime = field(default_factory=datetime.now)

@dataclass
class ProcessingLog:
    timestamp: datetime
    agent: str
    message: str
    level: str = "INFO"

@dataclass
class AgentState:
    symbol: str
    processing_timestamp: datetime = field(default_factory=datetime.now)

    # Data from different agents
    market_data: Optional[MarketData] = None
    forecast_data: Optional[ForecastData] = None
    macro_data: Optional[MacroData] = None
    sentiment_data: Optional[SentimentData] = None
    trading_recommendation: Optional[TradingRecommendation] = None

    # Processing metadata
    processing_log: List[ProcessingLog] = field(default_factory=list)
    data_quality_score: float = 1.0

    def add_log(self, agent: str, message: str, level: str = "INFO"):
        log_entry = ProcessingLog(
            timestamp=datetime.now(),
            agent=agent,
            message=message,
            level=level
        )
        self.processing_log.append(log_entry)

    def get_processing_summary(self):
        return {
            "agents_executed": len(set(log.agent for log in self.processing_log)),
            "errors": len([log for log in self.processing_log if log.level == "ERROR"]),
            "warnings": len([log for log in self.processing_log if log.level == "WARNING"]),
            "total_logs": len(self.processing_log)
        }

def create_empty_state(symbol: str) -> AgentState:
    return AgentState(symbol=symbol.upper())

# =============================================================================
# AGENT 1: MARKET DATA NODE
# =============================================================================

class MarketDataAgent:
    def __init__(self):
        self.name = "MarketDataAgent"
        print(f"✅ {self.name} initialized")

    async def process(self, state: AgentState, period: str = "6mo") -> AgentState:
        try:
            print(f"📊 {self.name}: Fetching market data for {state.symbol}...")

            # Fetch data from Yahoo Finance
            ticker = yf.Ticker(state.symbol)
            data = ticker.history(period=period)

            if data.empty:
                raise ValueError(f"No data available for {state.symbol}")

            # Calculate metrics
            current_price = float(data['Close'].iloc[-1])
            prices = data['Close']
            volume = data['Volume']

            # Returns
            returns = prices.pct_change()
            return_1d = float(returns.iloc[-1]) if len(returns) > 0 else 0.0
            return_5d = float(returns.tail(5).mean()) if len(returns) >= 5 else 0.0
            return_20d = float(returns.tail(20).mean()) if len(returns) >= 20 else 0.0

            # Volatility
            volatility_20d = float(returns.tail(20).std() * np.sqrt(252)) if len(returns) >= 20 else 0.0

            # Trend analysis
            if len(prices) >= 10:
                recent_avg = prices.tail(5).mean()
                older_avg = prices.tail(15).head(10).mean()
                if recent_avg > older_avg * 1.03:
                    trend = "bullish"
                elif recent_avg < older_avg * 0.97:
                    trend = "bearish"
                else:
                    trend = "neutral"
            else:
                trend = "neutral"

            # RSI calculation
            if len(returns) >= 14:
                gains = returns.where(returns > 0, 0).rolling(14).mean()
                losses = -returns.where(returns < 0, 0).rolling(14).mean()
                rs = gains / losses
                rsi = float(100 - (100 / (1 + rs.iloc[-1]))) if not np.isnan(rs.iloc[-1]) else 50.0
            else:
                rsi = 50.0

            # Support and resistance levels
            high_20 = prices.tail(20).max()
            low_20 = prices.tail(20).min()
            support_level = float(low_20 * 1.02)
            resistance_level = float(high_20 * 0.98)

            # Create market data object
            market_data = MarketData(
                symbol=state.symbol,
                current_price=current_price,
                prices=prices,
                volume=volume,
                rsi=rsi,
                trend=trend,
                return_1d=return_1d,
                return_5d=return_5d,
                return_20d=return_20d,
                volatility_20d=volatility_20d,
                support_level=support_level,
                resistance_level=resistance_level
            )

            state.market_data = market_data
            state.add_log(self.name, f"Market data loaded: ${current_price:.2f}, trend: {trend}")

            print(f"✅ {self.name}: Loaded {len(prices)} data points, price: ${current_price:.2f}")

        except Exception as e:
            print(f"❌ {self.name}: Error - {e}")
            state.add_log(self.name, f"Error: {str(e)}", "ERROR")

        return state

# =============================================================================
# AGENT 2: FORECASTING NODE
# =============================================================================

class ForecastingAgent:
    def __init__(self):
        self.name = "ForecastingAgent"
        print(f"✅ {self.name} initialized")

    async def process(self, state: AgentState, forecast_horizon: int = 5) -> AgentState:
        if not state.market_data or state.market_data.prices.empty:
            state.add_log(self.name, "No market data available", "ERROR")
            return state

        try:
            print(f"🔮 {self.name}: Generating forecasts for {state.symbol}...")

            prices = state.market_data.prices
            current_price = state.market_data.current_price

            # 1. Simple ARIMA-like forecast (moving average with trend)
            arima_forecast = self._arima_forecast(prices, current_price)

            # 2. Prophet forecast (if available)
            prophet_forecast = self._prophet_forecast(prices, forecast_horizon) if HAS_PROPHET else arima_forecast * 1.02

            # 3. LSTM-like forecast (simplified neural network simulation)
            lstm_forecast = self._lstm_forecast(prices, current_price)

            # 4. Ensemble forecast
            forecasts = [arima_forecast, prophet_forecast, lstm_forecast]
            ensemble_forecast = np.mean(forecasts)

            # 5. Calculate confidence
            forecast_std = np.std(forecasts)
            volatility = state.market_data.volatility_20d
            confidence = max(0.3, min(0.9, 1.0 - (forecast_std / current_price) - volatility))

            # 6. Prediction interval
            lower_bound = ensemble_forecast - 1.96 * forecast_std
            upper_bound = ensemble_forecast + 1.96 * forecast_std

            # 7. Accuracy score (based on volatility and trend consistency)
            trend_consistency = self._calculate_trend_consistency(prices)
            accuracy_score = confidence * trend_consistency

            forecast_data = ForecastData(
                arima_forecast=arima_forecast,
                prophet_forecast=prophet_forecast,
                lstm_forecast=lstm_forecast,
                ensemble_forecast=ensemble_forecast,
                forecast_confidence=confidence,
                prediction_interval=[lower_bound, upper_bound],
                forecast_horizon_days=forecast_horizon,
                forecast_accuracy_score=accuracy_score
            )

            state.forecast_data = forecast_data
            state.add_log(self.name, f"Forecasts generated: ${ensemble_forecast:.2f} (confidence: {confidence:.1%})")

            print(f"✅ {self.name}: Ensemble forecast: ${ensemble_forecast:.2f}, confidence: {confidence:.1%}")

        except Exception as e:
            print(f"❌ {self.name}: Error - {e}")
            state.add_log(self.name, f"Error: {str(e)}", "ERROR")

        return state

    def _arima_forecast(self, prices: pd.Series, current_price: float) -> float:
        if len(prices) >= 20:
            # Simple trend + moving average
            ma_5 = prices.tail(5).mean()
            ma_20 = prices.tail(20).mean()
            trend_factor = (ma_5 - ma_20) / ma_20
            return current_price * (1 + trend_factor * 0.5)
        return current_price * 1.01

    def _prophet_forecast(self, prices: pd.Series, horizon: int) -> float:
        if not HAS_PROPHET or len(prices) < 30:
            return prices.iloc[-1] * 1.02

        try:
            # Prepare data for Prophet
            df = pd.DataFrame({
                'ds': prices.index,
                'y': prices.values
            })

            # Create and fit model
            model = Prophet(daily_seasonality=True, yearly_seasonality=False)
            model.fit(df)

            # Make forecast
            future = model.make_future_dataframe(periods=horizon)
            forecast = model.predict(future)

            return float(forecast['yhat'].iloc[-1])
        except:
            return prices.iloc[-1] * 1.02

    def _lstm_forecast(self, prices: pd.Series, current_price: float) -> float:
        # Simulate LSTM behavior with exponential smoothing
        if len(prices) >= 10:
            weights = np.exp(np.linspace(-1, 0, min(10, len(prices))))
            weights = weights / weights.sum()
            recent_prices = prices.tail(len(weights))
            weighted_avg = np.sum(recent_prices * weights)

            # Add some volatility-adjusted randomness
            volatility = prices.pct_change().std()
            noise = np.random.normal(0, volatility * 0.1)
            return weighted_avg * (1 + noise)
        return current_price * (1 + np.random.normal(0.02, 0.05))

    def _calculate_trend_consistency(self, prices: pd.Series) -> float:
        if len(prices) < 10:
            return 0.5

        # Check how consistent the trend is
        returns = prices.pct_change().dropna()
        if len(returns) == 0:
            return 0.5

        # Measure trend consistency
        positive_days = (returns > 0).sum()
        negative_days = (returns < 0).sum()
        total_days = len(returns)

        # Higher consistency = more predictable
        consistency = 1 - (min(positive_days, negative_days) / total_days)
        return max(0.3, min(0.9, consistency))

# =============================================================================
# AGENT 3: MACRO ECONOMIC NODE
# =============================================================================

class MacroEconomicAgent:
    def __init__(self):
        self.name = "MacroEconomicAgent"
        print(f"✅ {self.name} initialized")

    async def process(self, state: AgentState) -> AgentState:
        try:
            print(f"🌍 {self.name}: Analyzing macro-economic factors...")

            # Simulate macro data (in real implementation, fetch from FRED, etc.)
            macro_data = MacroData(
                gdp_growth=np.random.normal(2.5, 0.5),  # ~2.5% GDP growth
                inflation_rate=np.random.normal(3.2, 0.3),  # ~3.2% inflation
                unemployment_rate=np.random.normal(3.8, 0.2),  # ~3.8% unemployment
                federal_funds_rate=np.random.normal(5.25, 0.25),  # ~5.25% fed rate
                vix=np.random.normal(18, 5),  # VIX around 18
                dollar_index=np.random.normal(103, 2),  # DXY around 103
                market_sentiment=np.random.choice(["bullish", "neutral", "bearish"], p=[0.3, 0.4, 0.3])
            )

            state.macro_data = macro_data
            state.add_log(self.name, f"Macro analysis: {macro_data.market_sentiment} sentiment")

            print(f"✅ {self.name}: Market sentiment: {macro_data.market_sentiment}")

        except Exception as e:
            print(f"❌ {self.name}: Error - {e}")
            state.add_log(self.name, f"Error: {str(e)}", "ERROR")

        return state

# =============================================================================
# AGENT 4: SENTIMENT ANALYSIS NODE
# =============================================================================

class SentimentAgent:
    def __init__(self):
        self.name = "SentimentAgent"
        print(f"✅ {self.name} initialized")

    async def process(self, state: AgentState) -> AgentState:
        try:
            print(f"😊 {self.name}: Analyzing market sentiment for {state.symbol}...")

            # Simulate sentiment analysis (in real implementation, use news APIs, etc.)
            news_sentiment = np.random.normal(0.1, 0.3)  # Slightly positive bias
            social_sentiment = np.random.normal(0.0, 0.4)  # More volatile

            overall_sentiment = (news_sentiment + social_sentiment) / 2

            # Determine sentiment trend
            if overall_sentiment > 0.2:
                sentiment_trend = "positive"
            elif overall_sentiment < -0.2:
                sentiment_trend = "negative"
            else:
                sentiment_trend = "neutral"

            # Simulate key topics
            topics = ["earnings", "product_launch", "market_conditions", "regulation", "competition"]
            key_topics = np.random.choice(topics, size=np.random.randint(1, 4), replace=False).tolist()

            sentiment_data = SentimentData(
                news_sentiment=news_sentiment,
                social_media_sentiment=social_sentiment,
                overall_sentiment=overall_sentiment,
                sentiment_trend=sentiment_trend,
                confidence_score=np.random.uniform(0.6, 0.9),
                key_topics=key_topics
            )

            state.sentiment_data = sentiment_data
            state.add_log(self.name, f"Sentiment analysis: {sentiment_trend} ({overall_sentiment:.2f})")

            print(f"✅ {self.name}: Overall sentiment: {sentiment_trend} ({overall_sentiment:.2f})")

        except Exception as e:
            print(f"❌ {self.name}: Error - {e}")
            state.add_log(self.name, f"Error: {str(e)}", "ERROR")

        return state

# =============================================================================
# AGENT 5: AI TRADING STRATEGIST
# =============================================================================

class AITradingStrategist:
    def __init__(self, api_key: Optional[str] = None):
        self.name = "AITradingStrategist"
        self.client = None
        if api_key and HAS_OPENAI:
            self.client = OpenAI(api_key=api_key)
        print(f"✅ {self.name} initialized {'with OpenAI' if self.client else 'with rule-based logic'}")

    async def process(self, state: AgentState) -> AgentState:
        try:
            print(f"🧠 {self.name}: Generating trading recommendation for {state.symbol}...")

            if self.client:
                recommendation = await self._generate_ai_recommendation(state)
            else:
                recommendation = self._generate_rule_based_recommendation(state)

            state.trading_recommendation = recommendation
            state.add_log(self.name, f"Recommendation: {recommendation.action} ({recommendation.confidence:.1%})")

            print(f"✅ {self.name}: {recommendation.action} recommendation with {recommendation.confidence:.1%} confidence")

        except Exception as e:
            print(f"❌ {self.name}: Error - {e}")
            state.add_log(self.name, f"Error: {str(e)}", "ERROR")

        return state

    async def _generate_ai_recommendation(self, state: AgentState) -> TradingRecommendation:
        # Create context for AI
        context = self._build_context(state)

        prompt = f"""You are an expert financial advisor. Based on the following data, provide a trading recommendation.

{context}

Provide your recommendation in this exact format:
ACTION: [BUY/SELL/HOLD]
CONFIDENCE: [0.0-1.0]
POSITION_SIZE: [0.0-1.0]
REASONING: [Brief explanation]
RISK_LEVEL: [LOW/MEDIUM/HIGH]
TIME_HORIZON: [SHORT/MEDIUM/LONG]
"""

        try:
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7,
                max_tokens=500
            )

            return self._parse_ai_response(response.choices[0].message.content, state)
        except:
            # Fallback to rule-based
            return self._generate_rule_based_recommendation(state)

    def _generate_rule_based_recommendation(self, state: AgentState) -> TradingRecommendation:
        current_price = state.market_data.current_price if state.market_data else 100

        # Initialize scoring
        buy_score = 0
        sell_score = 0
        confidence_factors = []

        # Market data analysis
        if state.market_data:
            md = state.market_data

            # Trend analysis
            if md.trend == "bullish":
                buy_score += 2
                confidence_factors.append("bullish_trend")
            elif md.trend == "bearish":
                sell_score += 2
                confidence_factors.append("bearish_trend")

            # RSI analysis
            if md.rsi < 30:  # Oversold
                buy_score += 2
                confidence_factors.append("oversold_rsi")
            elif md.rsi > 70:  # Overbought
                sell_score += 2
                confidence_factors.append("overbought_rsi")

            # Volatility analysis
            if md.volatility_20d > 0.4:  # High volatility
                sell_score += 1
                confidence_factors.append("high_volatility")

        # Forecast analysis
        if state.forecast_data:
            fd = state.forecast_data
            price_change = (fd.ensemble_forecast - current_price) / current_price

            if price_change > 0.05:  # >5% upside
                buy_score += 3
                confidence_factors.append("positive_forecast")
            elif price_change < -0.05:  # >5% downside
                sell_score += 3
                confidence_factors.append("negative_forecast")

            # Confidence in forecast
            if fd.forecast_confidence > 0.7:
                buy_score += 1 if price_change > 0 else 0
                sell_score += 1 if price_change < 0 else 0

        # Sentiment analysis
        if state.sentiment_data:
            sd = state.sentiment_data
            if sd.overall_sentiment > 0.3:
                buy_score += 1
            elif sd.overall_sentiment < -0.3:
                sell_score += 1

        # Macro analysis
        if state.macro_data:
            md = state.macro_data
            if md.market_sentiment == "bullish":
                buy_score += 1
            elif md.market_sentiment == "bearish":
                sell_score += 1

        # Make decision
        if buy_score > sell_score and buy_score >= 3:
            action = "BUY"
            confidence = min(0.9, 0.5 + (buy_score - sell_score) * 0.1)
            position_size = min(0.5, confidence * 0.6)
            risk_level = "MEDIUM" if confidence > 0.7 else "HIGH"
        elif sell_score > buy_score and sell_score >= 3:
            action = "SELL"
            confidence = min(0.9, 0.5 + (sell_score - buy_score) * 0.1)
            position_size = min(0.3, confidence * 0.4)
            risk_level = "MEDIUM" if confidence > 0.7 else "HIGH"
        else:
            action = "HOLD"
            confidence = 0.6
            position_size = 0.0
            risk_level = "LOW"

        # Calculate entry, stop loss, and take profit
        entry_price = current_price

        if action == "BUY":
            stop_loss = current_price * 0.95  # 5% stop loss
            take_profit = current_price * 1.15  # 15% take profit
        elif action == "SELL":
            stop_loss = current_price * 1.05  # 5% stop loss (short)
            take_profit = current_price * 0.85  # 15% take profit (short)
        else:
            stop_loss = current_price * 0.98
            take_profit = current_price * 1.05

        reasoning = f"Rule-based analysis: Buy score: {buy_score}, Sell score: {sell_score}. "
        reasoning += f"Key factors: {', '.join(confidence_factors[:3])}."

        return TradingRecommendation(
            action=action,
            confidence=confidence,
            position_size=position_size,
            entry_price=entry_price,
            stop_loss=stop_loss,
            take_profit=take_profit,
            risk_level=risk_level,
            reasoning=reasoning,
            time_horizon="MEDIUM"
        )

    def _build_context(self, state: AgentState) -> str:
        context = f"SYMBOL: {state.symbol}\n"

        if state.market_data:
            md = state.market_data
            context += f"MARKET DATA:\n"
            context += f"- Current Price: ${md.current_price:.2f}\n"
            context += f"- Trend: {md.trend}\n"
            context += f"- RSI: {md.rsi:.1f}\n"
            context += f"- 1D Return: {md.return_1d:.2%}\n"
            context += f"- Volatility: {md.volatility_20d:.1%}\n\n"

        if state.forecast_data:
            fd = state.forecast_data
            context += f"FORECAST DATA:\n"
            context += f"- Ensemble Forecast: ${fd.ensemble_forecast:.2f}\n"
            context += f"- Confidence: {fd.forecast_confidence:.1%}\n"
            context += f"- Price Change: {((fd.ensemble_forecast - state.market_data.current_price) / state.market_data.current_price):.1%}\n\n"

        if state.sentiment_data:
            sd = state.sentiment_data
            context += f"SENTIMENT DATA:\n"
            context += f"- Overall Sentiment: {sd.overall_sentiment:.2f}\n"
            context += f"- Trend: {sd.sentiment_trend}\n\n"

        return context

    def _parse_ai_response(self, response: str, state: AgentState) -> TradingRecommendation:
        # Parse AI response (simplified)
        lines = response.strip().split('\n')
        action = "HOLD"
        confidence = 0.5
        position_size = 0.1
        reasoning = "AI-generated recommendation"
        risk_level = "MEDIUM"
        time_horizon = "MEDIUM"

        for line in lines:
            if line.startswith("ACTION:"):
                action = line.split(":")[1].strip()
            elif line.startswith("CONFIDENCE:"):
                confidence = float(line.split(":")[1].strip())
            elif line.startswith("POSITION_SIZE:"):
                position_size = float(line.split(":")[1].strip())
            elif line.startswith("REASONING:"):
                reasoning = line.split(":", 1)[1].strip()
            elif line.startswith("RISK_LEVEL:"):
                risk_level = line.split(":")[1].strip()
            elif line.startswith("TIME_HORIZON:"):
                time_horizon = line.split(":")[1].strip()

        current_price = state.market_data.current_price if state.market_data else 100

        return TradingRecommendation(
            action=action,
            confidence=confidence,
            position_size=position_size,
            entry_price=current_price,
            stop_loss=current_price * (0.95 if action == "BUY" else 1.05),
            take_profit=current_price * (1.15 if action == "BUY" else 0.85),
            risk_level=risk_level,
            reasoning=reasoning,
            time_horizon=time_horizon
        )

In [21]:
class AIFinancialOrchestrator:
    def __init__(self, openai_api_key: Optional[str] = None):
        self.agents = {
            'market_data': MarketDataAgent(),
            'forecasting': ForecastingAgent(),
            'macro': MacroEconomicAgent(),
            'sentiment': SentimentAgent(),
            'strategist': AITradingStrategist(openai_api_key)
        }
        print(f"🎯 AIFinancialOrchestrator initialized with {len(self.agents)} agents")

    async def analyze_symbol(self, symbol: str, full_analysis: bool = True) -> AgentState:
        """Run complete analysis on a symbol"""

        print(f"\n🚀 STARTING ANALYSIS FOR {symbol}")
        print("=" * 60)

        # Create initial state
        state = create_empty_state(symbol)

        # Run agents in sequence
        try:
            # 1. Market Data (required)
            state = await self.agents['market_data'].process(state)

            if not state.market_data:
                print(f"❌ Cannot proceed without market data for {symbol}")
                return state

            if full_analysis:
                # 2. Forecasting
                state = await self.agents['forecasting'].process(state)

                # 3. Macro Economic Analysis
                state = await self.agents['macro'].process(state)

                # 4. Sentiment Analysis
                state = await self.agents['sentiment'].process(state)

                # 5. AI Trading Strategy
                state = await self.agents['strategist'].process(state)

            print(f"\n✅ ANALYSIS COMPLETE FOR {symbol}")
            print("=" * 60)
            return state

        except Exception as e:
            print(f"❌ Orchestrator Error: {str(e)}")
            state.add_log("Orchestrator", f"Error during analysis: {str(e)}", "ERROR")
            return state

In [29]:
# Instantiate the orchestrator and run the analysis
orchestrator = AIFinancialOrchestrator(openai_api_key=os.environ.get('OPENAI_API_KEY'))

async def run_analysis():
    # Run analysis for a symbol (e.g., "AAPL")
    analysis_result = await orchestrator.analyze_symbol("PLTR")

    # You can now inspect the analysis_result object
    print("\n--- Analysis Summary ---")
    if analysis_result.market_data:
        print(f"Symbol: {analysis_result.symbol}")
        print(f"Current Price: ${analysis_result.market_data.current_price:.2f}")
        print(f"Trend: {analysis_result.market_data.trend}")
    if analysis_result.forecast_data:
        print(f"Ensemble Forecast: ${analysis_result.forecast_data.ensemble_forecast:.2f}")
        print(f"Forecast Confidence: {analysis_result.forecast_data.forecast_confidence:.1%}")
    if analysis_result.trading_recommendation:
        rec = analysis_result.trading_recommendation
        print(f"Recommendation: {rec.action} (Confidence: {rec.confidence:.1%})")
        print(f"Reasoning: {rec.reasoning}")

# Run the async function
import asyncio
await run_analysis()

✅ MarketDataAgent initialized
✅ ForecastingAgent initialized
✅ MacroEconomicAgent initialized
✅ SentimentAgent initialized
✅ AITradingStrategist initialized with OpenAI
🎯 AIFinancialOrchestrator initialized with 5 agents

🚀 STARTING ANALYSIS FOR PLTR
📊 MarketDataAgent: Fetching market data for PLTR...
✅ MarketDataAgent: Loaded 125 data points, price: $174.03
🔮 ForecastingAgent: Generating forecasts for PLTR...
✅ ForecastingAgent: Ensemble forecast: $179.76, confidence: 54.7%
🌍 MacroEconomicAgent: Analyzing macro-economic factors...
✅ MacroEconomicAgent: Market sentiment: neutral
😊 SentimentAgent: Analyzing market sentiment for PLTR...
✅ SentimentAgent: Overall sentiment: neutral (-0.08)
🧠 AITradingStrategist: Generating trading recommendation for PLTR...
✅ AITradingStrategist: HOLD recommendation with 60.0% confidence

✅ ANALYSIS COMPLETE FOR PLTR

--- Analysis Summary ---
Symbol: PLTR
Current Price: $174.03
Trend: bullish
Ensemble Forecast: $179.76
Forecast Confidence: 54.7%
Recommend

In [31]:
# Create your own financial goal
my_goal = FinancialGoal(
    target_amount=200_000,      # Your $200K goal
    current_amount=5_000,       # Starting amount
    monthly_contribution=3_000, # Monthly investment capacity
    time_horizon_years=10,      # Timeline
    risk_tolerance="moderate"   # Risk level
)

# Create plan
orchestrator = AIFinancialOrchestrator()
result = await orchestrator.create_financial_plan(my_goal)

NameError: name 'FinancialGoal' is not defined

In [32]:
import sys
import importlib.util
import asyncio

# Define the path to the finance planner file
file_path = '/content/finance_planner_node__2_.py' # Use the renamed filename

# Load the module explicitly
spec = importlib.util.spec_from_file_location("finance_planner_module", file_path)
if spec is None:
    print(f"❌ Error: Could not find module at {file_path}")
else:
    finance_planner_module = importlib.util.module_from_spec(spec)
    sys.modules["finance_planner_module"] = finance_planner_module
    try:
        spec.loader.exec_module(finance_planner_module)

        # Run the finance planner's main function
        if hasattr(finance_planner_module, 'main'):
            print("✅ Finance planner module loaded successfully. Running main function...")
            await finance_planner_module.main()
        else:
            print(f"❌ Error: 'main' function not found in {file_path}")

    except Exception as e:
        print(f"❌ Error executing finance planner module: {e}")

❌ Error executing finance planner module: [Errno 2] No such file or directory: '/content/finance_planner_node__2_.py'
